In [ ]:
! pip install joblib
! pip install xgboost

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from influxdb import InfluxDBClient
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import joblib

In [ ]:
class PredictionModel:
    def __init__(self):
        self.X_train, self.y_train = [], []
        self.X_test, self.y_test = [], []
        self.model = None

        # Configuration
        self.PE_GAS = 1.00
        self.PE_ELEC = 2.17
        self.COP_H = 0.98
        self.EER_C = 5.4
        self.group_time = '1h'  # Grouping interval for resampling

    def load_from_influx(self, client, measurement, start, end):
        """
        Retrieves data from InfluxDB and prepares it for the ML model.
        """
        # Query the specific FMU fields
        influxql_query = f'''
        SELECT last("value") AS "val"
        FROM "{measurement}"
        WHERE time >= '{start}'
            AND time < '{end}'
            AND "name" =~ /^(DistrictHeating|DistrictCooling|Electricity|Tair_z1|Tair_z2|Tair_z3|Tair_z4|T_ext|DNI)$/
        GROUP BY time({self.group_time}), "name"
        fill(previous)
        '''

        # Query Data and Process into DataFrame
        result = client.query(influxql_query)
        points = result.get_points()
        
        df = pd.DataFrame(points)

        if df.empty:
            print("No data returned.")
            return

        # Convert 'time' to datetime
        df['time'] = pd.to_datetime(df['time'])

        # Pivot the DataFrame to have separate columns for each 'name'
        df = df.pivot_table(
            index='time',
            columns='name',
            values='val'
        ).reset_index()
        
        # Unit Conversion: data is in Joules, convert to kWh (J / 3,600,000)
        energy_fields = ['Electricity', 'DistrictHeating', 'DistrictCooling']
        for field in energy_fields:
            if field in df.columns:
                df[field] = df[field] / 3600000 

        # Add Temporal Features
        df['hour'] = df['time'].dt.hour
        df['is_weekend'] = df['time'].dt.dayofweek.isin([5, 6]).astype(int)
        
        return df

    def split(self, dataset, train_ratio):
        """
        Prepares features and targets. Splits into training and testing sets.
        """
        # Features based on building sensors
        X_cols = ['T_ext', 'DNI', 'Tair_z1', 'Tair_z2', 'Tair_z3', 'Tair_z4', 'hour', 'is_weekend']
        X = dataset[X_cols]

        # Target: Individual components [Electricity, Heating, Cooling]
        y = dataset[['Electricity', 'DistrictHeating', 'DistrictCooling']]
    
        # Split into training and testing sets
        split_point = int(len(X) * train_ratio)
        self.X_train, self.y_train = X.iloc[:split_point], y.iloc[:split_point]
        self.X_test, self.y_test = X.iloc[split_point:], y.iloc[split_point:]

    def compute_total_primary_energy(self, df):
        """
        Computes Total Primary Energy from individual components.
        """
        df['Total_Primary_Energy'] = (
            (df['DistrictHeating'] / self.COP_H * self.PE_GAS) + 
            (df['DistrictCooling'] / self.EER_C * self.PE_ELEC) + 
            (df['Electricity'] * self.PE_ELEC)
        )
        return df

    def train(self):
        """Trains the XGBoost model."""
        model = xgb.XGBRegressor(
            n_estimators=1000,
            learning_rate=0.05,
            max_depth=5,
            subsample=0.8,
            colsample_bytree=0.8,
            objective='reg:squarederror'
        )
        self.model = MultiOutputRegressor(model)
        self.model.fit(self.X_train, self.y_train)
        print("Model trained on InfluxDB data")

    def get_test_predictions(self):
        """
        Calculates the actual test values and corresponding predictions.
        
        Raises:
            ValueError: If the model is not trained or test data is missing.

        Returns:
            y_pred: np.array of predicted values for the test set.
        """
        if self.model is None:
            raise ValueError("Model is not trained. Please run the train method first.")
        if self.X_test.empty or self.y_test.empty:
            raise ValueError("Test data is not available. Please run the split method first.")
            
        # Make predictions on the unseen test set
        y_pred = self.model.predict(self.X_test)
        return y_pred
    
    def test_and_evaluate(self):
        """
        Evaluates the model.
        """
        try:
            predictions = self.get_test_predictions()
        except ValueError as e:
            print(f"Plotting Error: {e}")
            return
        
        print(f"\n--- Evaluation Results ---")

        targets = ['Electricity', 'DistrictHeating', 'DistrictCooling']
        df_pred = pd.DataFrame(predictions, columns=targets, index=self.y_test.index)
        df_actual = self.y_test.copy()

        df_pred = self.compute_total_primary_energy(df_pred)
        df_actual = self.compute_total_primary_energy(df_actual)

        all_targets = targets + ['Total_Primary_Energy']
        
        # Evaluate individual components
        for i, target_name in enumerate(all_targets):
            mse = mean_squared_error(self.y_test.iloc[:, i], predictions[:, i])
            r2 = r2_score(self.y_test.iloc[:, i], predictions[:, i])
            print(f"{target_name} -> MSE: {mse:.2f}, R²: {r2:.4f}")
        
    def save_model(self, filepath):
        """
        Saves the trained XGBoost model to a file using joblib for persistent storage.

        Args:
            filepath (str): The complete path and filename for saving the model.

        Returns:
            None: Prints a success or error message.
        """
        if self.model is None:
            print("ERROR: Model is not trained. Cannot save.")
            return

        try:
            # Save the model object
            joblib.dump(self.model, filepath)
            print(f"Model successfully saved to: {filepath}")
        except Exception as e:
            print(f"ERROR: Failed to save model: {e}")
    
    def load_model(self, filepath):
        """
        Loads a trained XGBoost model from a file using joblib.

        Args:
            filepath (str): The complete path and filename of the model file.

        Returns:
            None: Updates the internal self.model attribute. Prints an error message.
        """
        try:
            # Load the model object
            self.model = joblib.load(filepath)
        except FileNotFoundError:
            print(f"ERROR: Model file not found at: {filepath}")
        except Exception as e:
            print(f"ERROR: Failed to load model: {e}")

    # --- Plotting Methods ---

    def plot_feature_importance(self):
        """
        Plots Feature Importance.
        """
        # Plot importance for each internal estimator
        targets = ['Electricity', 'DistrictHeating', 'DistrictCooling']
        for i, target_name in enumerate(targets):
            plt.figure(figsize=(10, 6))
            # Access the individual XGBoost model for this specific target
            individual_model = self.model.estimators_[i]
            xgb.plot_importance(individual_model, importance_type='gain')
            plt.title(f"Feature Importance: {target_name}")
            plt.show()

    def plot_scatter_error(self):
        """
        Plots Predicted vs Actual values for all targets and Total Primary Energy,
        colored by absolute error.
        """
        try:
            predictions = self.get_test_predictions()
        except ValueError as e:
            print(f"Plotting Error: {e}")
            return

        # Prepare DataFrames for easy calculation
        targets = ['Electricity', 'DistrictHeating', 'DistrictCooling']
        df_pred = pd.DataFrame(predictions, columns=targets, index=self.y_test.index)
        df_actual = self.y_test.copy()

        # Add Total Primary Energy to both
        df_pred = self.compute_total_primary_energy(df_pred)
        df_actual = self.compute_total_primary_energy(df_actual)
        
        all_targets = targets + ['Total_Primary_Energy']
        
        # Create Subplots
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        axes = axes.flatten()

        for i, col in enumerate(all_targets):
            actual = df_actual[col]
            pred = df_pred[col]
            errors = np.abs(pred - actual)

            sc = axes[i].scatter(actual, pred, c=errors, cmap="coolwarm", s=20, alpha=0.7)
            fig.colorbar(sc, ax=axes[i], label="Abs Error")
            
            # Identity line
            min_val = min(actual.min(), pred.min())
            max_val = max(actual.max(), pred.max())
            axes[i].plot([min_val, max_val], [min_val, max_val], color="black", linestyle='--', linewidth=1)
            
            axes[i].set_title(f"Actual vs Predicted: {col}")
            axes[i].set_xlabel("Actual (kWh or PE)")
            axes[i].set_ylabel("Predicted (kWh or PE)")
            axes[i].grid(True)
        
        plt.tight_layout()
        plt.show()

    def plot_error_vs_actual(self):
        """
        Plots the prediction residual error (Predicted - Actual) against the actual PV generation.
        This helps identify if the model's error is correlated with the magnitude of the target variable.
        """
        try:
            predictions = self.get_test_predictions()
        except ValueError as e:
            print(f"Plotting Error: {e}")
            return

        targets = ['Electricity', 'DistrictHeating', 'DistrictCooling']
        df_pred = pd.DataFrame(predictions, columns=targets, index=self.y_test.index)
        df_actual = self.y_test.copy()

        df_pred = self.compute_total_primary_energy(df_pred)
        df_actual = self.compute_total_primary_energy(df_actual)

        all_targets = targets + ['Total_Primary_Energy']
    
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        axes = axes.flatten()

        for i, col in enumerate(all_targets):
            actual = df_actual[col]
            residual = df_pred[col] - actual

            axes[i].scatter(actual, residual, s=15, alpha=0.6, color='tab:blue')
            axes[i].axhline(0, color="black", linewidth=1.5, linestyle='--')
            
            axes[i].set_title(f"Residuals: {col}")
            axes[i].set_xlabel("Actual Value")
            axes[i].set_ylabel("Error (Pred - Actual)")
            axes[i].grid(True)

        plt.tight_layout()
        plt.show()

In [ ]:
# InfluxDB Connection Details
INFLUX_HOST = 'influxdb'       # The host where InfluxDB is running
INFLUX_PORT = 8086              # The port
INFLUX_USER = 'admin'           # Username
INFLUX_PASS = 'admin123'        # Password
INFLUX_DB   = 'controlled_building' # Name of the BMS database

# Configuration
MEASUREMENT = "simulation_observations" # The measurement containing these fields
YEAR = 2025
# Full year
#START = f"{YEAR}-01-01T00:00:00Z"
#END = f"{YEAR + 1}-01-01T00:00:00Z"
# Summer
START = f"{YEAR}-06-01T00:00:00Z"
END = f"{YEAR}-09-01T00:00:00Z"

# Initialize the Client
client = InfluxDBClient(
    host=INFLUX_HOST,
    port=INFLUX_PORT,
    username=INFLUX_USER,
    password=INFLUX_PASS,
    database=INFLUX_DB
)

assert client.ping(), "❌ Cannot reach InfluxDB"
print("✅ Connected to InfluxDB 1.x")

In [ ]:
# Train the model
model = PredictionModel()
df_influx = model.load_from_influx(client, MEASUREMENT, START, END)
model.split(df_influx, 0.8)
model.train()

# Save the trained model
model.save_model("xgb_multioutput_model.joblib")

In [ ]:
# Load the trained model
# model.load_model("xgb_multioutput_model.joblib")

In [ ]:
# Evaluate the model
model.test_and_evaluate()
model.plot_feature_importance()
model.plot_scatter_error()
model.plot_error_vs_actual()